In [1]:
####### Run a simple analysis of terrain around a selected location

# Step 1: Sign up for a Google Earth Engine Account here:
# https://earthengine.google.com/signup/

# you will use this to authenticate your account with GEE to run the script and
# utilize GEE layers

In [ ]:
# get any missing modules
! pip install earthengine-api geemap

In [18]:
# setup modules
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import sys,os,os.path,tempfile

# spatial modules
import ee
#import geemap # currently not using as colorbars dont load properly in google collab
import geemap.colormaps as cm
import geemap.foliumap as geemap # use folium instead so coloramps work on goolge collab

# disable after updating
#geemap.update_package()

In [19]:
## setup output. defined dir or system temp dir
data_dir=os.path.expanduser(os.path.join(tempfile.gettempdir(),'terrain_analysis_output'))

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print(" ")
print("****** Raster and blockage results output folder ******")
print("****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******")
print("****** Use the dir navigation to go up one level, then navigate to tmp/geo ******")
print(data_dir)

 
****** Raster and blockage results output folder ******
****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******
****** Use the dir navigation to go up one level, then navigate to tmp/geo ******
/var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output


In [20]:
##### initialize ee API  - you need to authenticate with GEE and initialize

## Trigger the authentication flow. You only need to do this once while running notebook
ee.Authenticate()

## After inserting the API key initialize GEE
ee.Initialize()

In [21]:
### Define some GIS layers
# SEUS bounding box
bbox = ee.Geometry.Rectangle(-95.0, 29.5, -74.5, 36.5)

# some other layers - just examples, can add more
nexrad = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_locations').filterBounds(bbox)
AandM_EC_Tower = ee.FeatureCollection('users/serbinsh/vector_data/amf3/BNF_AandM_EC_Tower')
potential_BNF_AOS_sites = ee.FeatureCollection('users/serbinsh/vector_data/amf3/potential_BNF_AOS_sites')
AandM_EC_Tower_2kmbuff = ee.FeatureCollection('users/serbinsh/vector_data/amf3/BNF_AandM_EC_Tower_2kmbuff_AlbersEA')
nexrad = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_locations').filterBounds(bbox)
nexrad_100km = ee.FeatureCollection('users/serbinsh/vector_data/amf3/nexrad_100km_buffer_AEA').filterBounds(bbox)
neon_centroids = ee.FeatureCollection('users/serbinsh/vector_data/amf3/All_NEON_TOS_Plot_Points_V8').filterBounds(bbox)

In [22]:
# DEM - 1.3 Arc Second NED dataset
elev_dataset = ee.Image('USGS/NED').clip(bbox);
ned_elevation = elev_dataset.select('elevation')
dem_palette = cm.get_palette('Spectral_r', n_class=30)
demViz = {'min': 0.0, 'max': 2000.0, 'palette': dem_palette, 'opacity': 1}

# load NED 270m SRTM mTPI
mtpi_dataset = ee.Image('CSP/ERGo/1_0/US/mTPI').clip(bbox);
usMtpi = mtpi_dataset.select('elevation');
usMtpiVis = {'min': -200.0, 'max': 200.0, 'palette': ['0b1eff', '4be450', 'fffca4', 'ffa011', 'ff0000']};

# load 10m NED landforms
nedlandform_dataset = ee.Image('CSP/ERGo/1_0/US/landforms').clip(bbox);
landforms = nedlandform_dataset.select('constant');
landformsVis = {'min': 11.0, 'max': 42.0,'palette': [
    '141414', '383838', '808080', 'EBEB8F', 'F7D311', 'AA0000', 'D89382',
    'DDC9C9', 'DCCDCE', '1C6330', '68AA63', 'B5C98E', 'E1F0E5', 'a975ba',
    '6f198c']};

In [23]:
### create hillshade & slope
ned_hillshade = ee.Terrain.hillshade(ned_elevation)
ned_slope = ee.Terrain.slope(ned_elevation)
#ned_slope.getInfo()

In [42]:
### Show NED DEM data
srtm = geemap.Map(center=[33.315809,-85.198609], zoom=8)
srtm.addLayer(ned_elevation, demViz, 'Elevation above sea level');

srtm.addLayer(AandM_EC_Tower, {'color': 'black', 'strokeWidth': 1}, 'A & M Tower')
srtm.addLayer(AandM_EC_Tower_2kmbuff, {'color': 'black', 'opacity': 0.4}, 'A & M EC tower 2km buffer')
srtm.addLayer(nexrad,{'color': 'yellow', 'strokeWidth': 3},'NEXRAD locations')
srtm.addLayer(nexrad_100km, {'color': '#d3d3d3', 'opacity': 0.20}, 'NEXRAD 100km buffers')

srtm.add_colorbar(colors=demViz['palette'], vmin=demViz['min'], 
                  vmax=demViz['max'], label="Elevation (m)", 
                  caption="Elevation (m)",
                  layer_name="SRTM DEM", position="bottomright")


states = ee.FeatureCollection('TIGER/2018/States')
statesImage = ee.Image().paint(states, 0, 2)
srtm.addLayer(statesImage, {'palette': 'black'}, 'US States')
srtm.addLayerControl()
srtm

In [41]:
### Show hillshade
hillshade = geemap.Map(center=[33.315809,-85.198609], zoom=8)
hillshade.addLayer(ned_hillshade, {min: 150, max:255}, 'Hillshade');

hillshade.addLayer(AandM_EC_Tower, {'color': 'black', 'strokeWidth': 1}, 'A & M Tower')
hillshade.addLayer(AandM_EC_Tower_2kmbuff, {'color': 'black', 'opacity': 0.4}, 'A & M EC tower 2km buffer')
hillshade.addLayer(nexrad,{'color': 'yellow', 'strokeWidth': 3},'NEXRAD locations')
hillshade.addLayer(nexrad_100km, {'color': '#d3d3d3', 'opacity': 0.20}, 'NEXRAD 100km buffers')

states = ee.FeatureCollection('TIGER/2018/States')
statesImage = ee.Image().paint(states, 0, 2)
hillshade.addLayer(statesImage, {'palette': 'black'}, 'US States')
hillshade.addLayerControl()
hillshade

In [40]:
### Show slope
slope = geemap.Map(center=[33.315809,-85.198609], zoom=8)
slope_palette = cm.get_palette('Spectral_r', n_class=30)
slopeViz = {'min': 0.0, 'max': 35, 'palette': slope_palette, 'opacity': 1}
slope.addLayer(ned_slope, slopeViz, 'Slope (degrees)');

slope.add_colorbar(colors=slopeViz['palette'], vmin=slopeViz['min'], 
                  vmax=slopeViz['max'], label="Slope (degrees)", 
                  caption="Slope (degrees)",
                  layer_name="SRTM Slope", position="bottomright")

slope.addLayer(AandM_EC_Tower, {'color': 'black', 'strokeWidth': 1}, 'A & M Tower')
slope.addLayer(AandM_EC_Tower_2kmbuff, {'color': 'black', 'opacity': 0.4}, 'A & M EC tower 2km buffer')
slope.addLayer(nexrad,{'color': 'yellow', 'strokeWidth': 3},'NEXRAD locations')
slope.addLayer(nexrad_100km, {'color': '#d3d3d3', 'opacity': 0.20}, 'NEXRAD 100km buffers')


slope

In [43]:
### Show NED DEM data w/ NEON sites. Zoome into TALL
srtm = geemap.Map(center=[32.9,-87.4], zoom=11)
srtm.addLayer(ned_elevation, demViz, 'Elevation above sea level');

srtm.addLayer(neon_centroids, {'color': 'black', 'strokeWidth': 1}, 'NEON Sampling Centroids')

srtm.add_colorbar(colors=demViz['palette'], vmin=demViz['min'], 
                  vmax=demViz['max'], label="Elevation (m)", 
                  caption="Elevation (m)",
                  layer_name="SRTM DEM", position="bottomright")


states = ee.FeatureCollection('TIGER/2018/States')
statesImage = ee.Image().paint(states, 0, 2)
srtm.addLayer(statesImage, {'palette': 'black'}, 'US States')
srtm.addLayerControl()
srtm

srtm

In [12]:
### Some stats - write to files.  Example CSV and KMZ
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
out_dem_stats = os.path.join(data_dir, 'nexrad_slope_stats.csv')
#geemap.zonal_statistics(ned_slope, nexrad_100km, out_dem_stats, statistics_type='MEAN', scale=1000)
geemap.zonal_statistics(ned_slope, nexrad_100km, out_dem_stats, statistics_type='MEAN')

out_dem_stats = os.path.join(data_dir, 'AandM_slope_mean.csv')
geemap.zonal_statistics(ned_slope, AandM_EC_Tower_2kmbuff, out_dem_stats, statistics_type='MEAN')
out_dem_stats = os.path.join(data_dir, 'AandM_slope_minmax.csv')
geemap.zonal_statistics(ned_slope, AandM_EC_Tower_2kmbuff, out_dem_stats, statistics_type='MIN_MAX')

# create output KMZ file of the buffer and the statistics embedded
out_dem_stats = os.path.join(data_dir, 'AandM_slope_mean.kmz')
geemap.zonal_statistics(ned_slope, AandM_EC_Tower_2kmbuff, out_dem_stats, statistics_type='MEAN')
out_dem_stats = os.path.join(data_dir, 'AandM_slope_minmax.kmz')
geemap.zonal_statistics(ned_slope, AandM_EC_Tower_2kmbuff, out_dem_stats, statistics_type='MIN_MAX')

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output/nexrad_slope_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output/AandM_slope_mean.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output/AandM_slope_minmax.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output/AandM_slope_mean.kmz
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/terrain_analysis_output/AandM_slope_minmax.kmz


In [13]:
### Generate stats to the screen - basic examples
# for some reason for AandM_EC_Tower_2kmbuff I get a different value for the mean slope
# some info
# https://colab.research.google.com/github/csaybar/EEwPython/blob/master/5_Reducer.ipynb

mn = ee.Reducer.mean()
sd = ee.Reducer.stdDev()
reducers = mn.combine(sd,"",True)

#Image.reduceRegion(reducer, geometry, scale, crs, crsTransform, bestEffort, maxPixels, tileScale)
dict = ned_slope.reduceRegion(reducers,geometry=AandM_EC_Tower_2kmbuff)
#print(ee.Dictionary(dict))
#print(ee.Dictionary(dict.getInfo()))
slope_vals = dict.getInfo()
#slope_vals["slope"]
print(slope_vals)


# large example - using ways to reduce complexity and approximate
dict = ned_slope.reduceRegion(reducers,geometry=nexrad_100km,bestEffort=True,maxPixels=10000000,tileScale=2)
#print(ee.Dictionary(dict))
#print(ee.Dictionary(dict.getInfo()))
slope_vals = dict.getInfo()
#slope_vals["slope"]
print(slope_vals)


# alternative
#print('Projection, crs, and crs_transform:', ee.Image('USGS/NED').select("B0").projection());
buff_proj = AandM_EC_Tower_2kmbuff.getInfo()
image_proj = ned_slope.projection().getInfo()
#print(image_proj)
print(ned_slope.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': AandM_EC_Tower_2kmbuff.geometry(),
  'crs': 'EPSG:4326', # LL WGS84
  'crsTransform': image_proj['transform'],
  'maxPixels': 1e9
}).getInfo()) 

{'slope_mean': 12.870619972514502, 'slope_stdDev': 7.9456466379441375}
{'slope_mean': 1.3713268414973405, 'slope_stdDev': 2.443205569272882}
{'slope': 12.87061982375478}


In [14]:
#print(ee.Feature(nexrad_100km.first().getInfo()))

In [15]:
### Generate stats to the screen - basic zonal stats by group
#nexrad_100km.getInfo()
MeansFeatures = ned_slope.reduceRegions(**{
    'collection': nexrad_100km,
    'reducer': ee.Reducer.mean(),
    'scale': 30,})

In [16]:
#print(ee.Feature(MeansFeatures.first()).select(ned_slope.bandNames()).getInfo())
ee.Feature(MeansFeatures.first()).getInfo()

#print(MeansFeatures.set(ee.Dictionary(ee.List(MeansFeatures).flatten())))
#feature.set(ee.Dictionary(ee.List(list).flatten()))
#print(ee.Dictionary((MeansFeatures.getInfo())))
#print('Count:', MeansFeatures.properties());

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-83.31357744904112, 35.00712193781416],
    [-83.31765161298728, 34.979237927708624],
    [-83.3206430201576, 34.95126255997807],
    [-83.32254219450364, 34.92322610736392],
    [-83.32335690172776, 34.89515603207209],
    [-83.32307926933207, 34.86707391410789],
    [-83.32171540215356, 34.839016019113885],
    [-83.31926841306391, 34.81100479389407],
    [-83.31573507785758, 34.783066782788865],
    [-83.3111283551545, 34.755229842365154],
    [-83.3054544614975, 34.72753036262109],
    [-83.29871238040296, 34.69998134078682],
    [-83.29091361254072, 34.67261946534564],
    [-83.28206634681295, 34.64546781245999],
    [-83.27217799100943, 34.61855378371609],
    [-83.26126701383036, 34.59190573007244],
    [-83.24934131181682, 34.565546713541025],
    [-83.23640906727982, 34.53949964042622],
    [-83.2224874932886, 34.51379717072569],
    [-83.20759548982163, 34.48846356608181],
    [-83.19174163133432, 34.463

In [17]:
#import pprint
#pprint.pprint(ned_slope.getInfo())
#print(MeansFeatures.get('properties'))
#filter = ee.Filter.inList('ADM1_NAME', ['Hoa Binh', 'Thanh Hoa']);